In [1]:
import requests
import time
import json
from tools import get_key

GENIUS_API_TOKEN = get_key("Client_access_genius")

headers = {
    'Authorization': f'Bearer {GENIUS_API_TOKEN}'
}

In [2]:
# Test de récupération de la liste de sons d'un artiste via l'API Genius
def get_artist_songs(artist_id, page=1):
    url = f'https://api.genius.com/artists/{artist_id}/songs'
    params = {
        'page': page,
        'sort': 'popularity'
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None

In [ ]:
def get_artist_featurings(artist_id, max_pages=None):
    base_url = f'https://api.genius.com/artists/{artist_id}/songs'
    page = 1
    featuring_songs = []

    while True:
        params = {
            'page': page,
            'sort': 'popularity'
        }
        response = requests.get(base_url, headers=headers, params=params)

        if response.status_code != 200:
            print(f"Erreur : {response.status_code}")
            break

        data = response.json()
        songs = data.get("response", {}).get("songs", [])

        for song in songs:
            title = song.get("title")
            artist_names = song.get("artist_names")
            # Filtre : on conserve uniquement si plusieurs artistes sont indiqués
            if ',' in artist_names or '&' in artist_names or 'feat' in artist_names.lower():
                featuring_songs.append((title, artist_names))

        # Pagination
        next_page = data.get("response", {}).get("next_page")
        if not next_page or (max_pages and page >= max_pages):
            break
        page += 1

    return featuring_songs


def get_artist_songs_with_authors(artist_id, max_pages=1):
    songs = []
    for page in range(1, max_pages + 1):
        url = f'https://api.genius.com/artists/{artist_id}/songs'
        params = {
            'page': page,
            'sort': 'popularity'
        }
        response = requests.get(url, headers=headers, params=params)
        if response.status_code != 200:
            print(f"Erreur page {page} : {response.status_code}")
            break

        data = response.json()
        song_list = data.get('response', {}).get('songs', [])

        for song in song_list:
            title = song.get('title')
            primary_artist = song.get('primary_artist', {}).get('name')
            featured_artists = song.get('featured_artists', [])

            # Extraire les noms des artistes (principal + feats)
            authors = [primary_artist] + [artist['name'] for artist in featured_artists]
            if len(authors) > 1:
                songs.append((title, authors))

        if not song_list:
            break

    return songs




In [21]:
songs = get_artist_featurings(13325, 100)
songs

[('\u200b\u200b\u200b\u200bavf', 'Stromae (Ft. GIMS & OrelSan)'),
 ('Mi Gna', 'GIMS (Ft. Spitakci Hayko & Super Sako)'),
 ('ON OFF', 'Shirin David & GIMS'),
 ('Sharingan', 'GIMS (Ft. Insolent, OrelSan & The Shin Sekaï)'),
 ('Corazón', 'GIMS (Ft. French Montana & Lil Wayne)'),
 ('Loin (Pilule Violette)', 'GIMS (Ft. Dany Synthé, Nius & SoFLY)'),
 ('Ma fierté', 'Dadju (Ft. Alonzo & GIMS)'),
 ('Ça décoiffe', 'GIMS (Ft. Black M & JR O Crom)'),
 ('Arhbo [Music from the FIFA World Cup Qatar 2022 Official Soundtrack]',
  'GIMS, Ozuna & RedOne (Ft. FIFA Sound)'),
 ('Laisse tomber', 'GIMS (Ft. Dr Bériz & Insolent)'),
 ('I love you', 'Dadju & Tayc'),
 ('SEYA', 'SativaMusic, Morad & GIMS'),
 ('Bella ciao', 'Naestro (Ft. Dadju, GIMS, Slimane & Vitaa)'),
 ('De Marseille à Paris', 'GIMS (Ft. Bedjik, Dr Bériz, H Magnum & Soprano)'),
 ('Boucan (Pilule Violette)', 'GIMS (Ft. DJ LastOne & JuL)'),
 ('GJS', 'GIMS (Ft. JuL & SCH)'),
 ('SPIDER', 'GIMS & DYSTINCT'),
 ('BELLE', 'GIMS (Ft. Dadju & Slimane)'),
 